In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import xlrd

# read excel sheets and remove these which are not relevant

xls = xlrd.open_workbook(r'Swiss_Powerlifting_Rekorde_2019.xlsx', on_demand=True)
sheets = xls.sheet_names()
sheets = [k for k in sheets if 'WUAP' not in k and 'PL EQU' not in k and 'Tabelle1' not in k
         and 'Equipped' not in k and 'TM' not in k]

# create dictionaries for categories, disciplines and gender

categories = ['Subjunior', 'Junior', 'Open', 'M1', 'M2', 'M3']
disciplines = ['Kniebeugen', 'Bankdrücken', 'Kreuzheben', 'Powerlifting Total', 'Single Lift Bankdrücken', 'Single Lift Kreuzheben']
              #'Equipped - Single Lift Bankdrücken', 'RAW Bankdrücken']
gender = ['Man', 'Woman']

In [2]:
records_df = pd.DataFrame()

# read in data

for sheetname in sheets:
    if records_df.empty:
        records_df = pd.read_excel('Swiss_Powerlifting_Rekorde_2019.xlsx', sheet_name=sheetname)
    else:
        df = pd.read_excel('Swiss_Powerlifting_Rekorde_2019.xlsx', sheet_name=sheetname)
        records_df = records_df.append(df, sort=False)
        
# change column names for easier filtering later

records_df = records_df.rename(columns={'National-Rekorde \nRecords nationaux\nClassic\nKniebeugen / Squat': 'Kniebeugen',
                              'National-Rekorde \nRecords nationaux\nClassic\nBankdrücken / Développé couché': 'Bankdrücken',
                              'National-Rekorde \nRecords nationaux\nClassic\nKreuzheben / Soulevé de terre': 'Kreuzheben',
                              'National-Rekorde \nRecords nationaux\nClassic\nPowerlifting Total' : 'Powerlifting Total',
                              'National-Rekorde \nRecords nationaux\nClassic - Single Lift\nBankdrücken / Développé couché' : 'Single Lift Bankdrücken',
                              'National-Rekorde \nRecords nationaux\nClassic - Single Lift\nKreuzheben / Soulevé de terre' : 'Single Lift Kreuzheben',
                              'National-Rekorde \nRecords nationaux\nEquipped - Single Lift\nBankdrücken / Développé couché' : 'Equipped - Single Lift Bankdrücken',
                              'National-Rekorde \nRecords nationaux\nRAW\nBankdrücken / Développé couché' : 'RAW Bankdrücken'})

In [3]:
# fixing index

records_df = records_df.reset_index()
records_df = records_df.drop(['index'], axis=1)

In [4]:
records_df['Disziplin'] = pd.Series()
records_df['Gewicht'] = pd.Series()

# mapping column names to disciplines and values from multiple columns to weight
# it creates sql-like table which can be grouped and sorted

for i in disciplines:
    records_df['Disziplin'].loc[~records_df[i].isnull()] = records_df[i].name
    for index, value in records_df[i].items():
        if str(value) != 'nan':
            records_df['Gewicht'].iloc[index] = value

records_df = records_df[records_df.columns.drop(list(disciplines))]
records_df.columns = ['Klasse', 'Athlet', 'Rekord', 'Datum', 'Wettkampf', 'Ort', 'Land', 'Disziplin', 'Gewicht']

/Users/mkot/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
records_df['Geschlecht'] = pd.Series()
records_df['Kategorie'] = pd.Series()

current_gender = ''
current_category = ''

# mapping gender and category rows to column values

for index, value in records_df['Klasse'].items():
    if value in gender:
        current_gender = value
    elif value in categories:
        current_category = value
        
    records_df['Geschlecht'].iloc[index] = current_gender
    records_df['Kategorie'].iloc[index] = current_category

In [6]:
# reducing the number of columns and reordering them

records_df = records_df[(records_df.Gewicht != 'Gewicht') & (records_df.Gewicht != 'Record') & (~records_df.Gewicht.isna())]
records_df = records_df.reset_index()
records_df = records_df.drop(['index'], axis=1)
records_df = records_df[['Kategorie', 'Disziplin', 'Geschlecht', 'Klasse', 'Athlet', 'Gewicht', 'Rekord', 'Datum', 'Wettkampf', 'Ort', 'Land']]

In [7]:
# fill NaNs with empty strings

records_df.fillna('', inplace=True)

In [8]:
# create special values for sorting

records_df['sort_order_kategorie'] = pd.Series()
records_df['sort_order_geschlecht'] = pd.Series()
records_df['sort_order_disziplin'] = pd.Series()

for index, row in records_df.iterrows():
    if row['Kategorie'] == 'Open':
        records_df.loc[index, 'sort_order_kategorie'] = 0
    elif row['Kategorie'] == 'Subjunior':
        records_df.loc[index, 'sort_order_kategorie'] = 1
    elif row['Kategorie'] == 'Junior':
        records_df.loc[index, 'sort_order_kategorie'] = 2
    elif row['Kategorie'] == 'M1':
        records_df.loc[index, 'sort_order_kategorie'] = 3
    elif row['Kategorie'] == 'M2':
        records_df.loc[index, 'sort_order_kategorie'] = 4
    elif row['Kategorie'] == 'M3':
        records_df.loc[index, 'sort_order_kategorie'] = 5
    
    if row['Geschlecht'] == 'Woman':
        records_df.loc[index, 'sort_order_geschlecht'] = 0
    else:
        records_df.loc[index, 'sort_order_geschlecht'] = 1
        
    if row['Disziplin'] == 'Kniebeugen':
        records_df.loc[index, 'sort_order_disziplin'] = 0
    elif row['Disziplin'] == 'Bankdrücken':
        records_df.loc[index, 'sort_order_disziplin'] = 1
    elif row['Disziplin'] == 'Kreuzheben':
        records_df.loc[index, 'sort_order_disziplin'] = 2
    elif row['Disziplin'] == 'Powerlifting Total':
        records_df.loc[index, 'sort_order_disziplin'] = 3
    elif row['Disziplin'] == 'Single Lift Bankdrücken':
        records_df.loc[index, 'sort_order_disziplin'] = 4
    elif row['Disziplin'] == 'Single Lift Kreuzheben':
        records_df.loc[index, 'sort_order_disziplin'] = 5
    elif row['Disziplin'] == 'Equipped - Single Lift Bankdrücken':
        records_df.loc[index, 'sort_order_disziplin'] = 6
    elif row['Disziplin'] == 'RAW Bankdrücken':
        records_df.loc[index, 'sort_order_disziplin'] = 7
        
# sort dataframe and remove special columns
        
records_df.sort_values(by = ['sort_order_kategorie', 'sort_order_disziplin', 'sort_order_geschlecht', 'Klasse'], inplace=True)
records_df = records_df.drop(['sort_order_kategorie', 'sort_order_geschlecht', 'sort_order_disziplin'], axis=1)

In [9]:
records_df

,Kategorie,Disziplin,Geschlecht,Klasse,Athlet,Gewicht,Rekord,Datum,Wettkampf,Ort,Land
38,Open,Kniebeugen,Woman,47,Sirman Coline,92.5,SR,2017,WEC,Luxemburg,LUX
39,Open,Kniebeugen,Woman,52,Keusch Katja,90,SR,2019,SM,Vilters,SUI
40,Open,Kniebeugen,Woman,57,Henzi Anna,132.5,SR,2020,SM,Dagmersellen,SUI
41,Open,Kniebeugen,Woman,63,Kamberi Jana,147.5,SR,2019,SM,Vilters,SUI
42,Open,Kniebeugen,Woman,72,Jonasch Anna,157.5,SR,2020,SM,Dagmersellen,SUI
43,Open,Kniebeugen,Woman,84,Butera Jessica,167.5,SR,2019,SM,Vilters,SUI
44,Open,Kniebeugen,Woman,84+,Butera Jessica,170,SR,2019,DSM,Zürich,SUI
30,Open,Kniebeugen,Man,59,Swiss standard,140,,,,,
31,Open,Kniebeugen,Man,66,Jost Alain,190,SR,2019,EM,Kaunas,LTU
32,Open,Kniebeugen,Man,74,Sigrist Sven,210,SR,2018,WM,Calgary,CAN


In [10]:
records_df.to_html(index=False, buf='records.html')